In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [15]:
#load movies list
movieList = pd.read_csv('movieList.csv')

In [16]:
url = "http://www.imdb.com/title/" + "tt0180093" + "/reviews"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def ParseMovie(movie_id):
    movieReviews = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 
                                           'movie_name', 'movie_id', 'numOfReviews',
                               'review_name', 'author', 'region', 'date', 'review_text'])
    
    
    #at first let's get info about the movie by id
    url = "http://www.imdb.com/title/" + movie_id + "/reviews"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    #get movie_name
    tn15title = soup.findChild(id="tn15title")
    movie_name = tn15title.find_all("a")[0].get_text()
    tn15content = soup.find(id="tn15content")
    numOfReviews = int(tn15content.find_all("table")[2].get_text().replace("\n"," ").split(" ")[3])
    print movie_name, numOfReviews, "---------------"
    #download reviews
    for k in xrange(numOfReviews/10+1):
        #create url
        start_review = 10*k
        url = "http://www.imdb.com/title/" + str(movie_id) + "/reviews?start=" + str(start_review)
        print  (1.0*start_review / numOfReviews)
        #download html
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        tn15content = soup.find(id="tn15content")

        #delete message about spoilers
        tn15_p = list(tn15content.find_all("p"))
        len1 = len(tn15_p)
        i=0
        while(i<len1):
            if tn15_p[i].get_text() == u'*** This review may contain spoilers ***':
                tn15_p.remove(tn15_p[i])
                i =-1
                len1 -= 1
            i+=1
        tn15_p.remove(tn15_p[len1-1])
        len1 -= 1 #number of review on the current page

        #preparate data for all the reviews on the page
        tn15_review = tn15content.find_all(class_='yn')

        tn15_small = tn15content.find_all("small")#useful, date, region
        review_name = tn15content.find_all("h2")
        tn15_div = tn15content.find_all("div")

        for i in xrange(len1):
            review = {}
            review['review_id'] = tn15_review[i].attrs['id'].split("_")[1]
            review['review_text'] = tn15_p[i].get_text().replace("\n", " ").lower()
            review['movie_name'] = movie_name
            review['movie_id'] = movie_id
            review['numOfReviews'] = numOfReviews
            review['review_name'] = review_name[i].get_text()
            review['author'] = tn15_div[2*i+0].find_all("a")[1].get_text()

            #check if we have rating
            if (len(tn15content.find_all('div')[2*i].find_all('img'))) == 1:
                review['rating'] = None
            else: 
                review['rating'] = tn15content.find_all("div")[2*i].find_all("img")[1].attrs['alt'].split("/")[0]


            #check if we have region and uselful
            tn15_small = tn15_div[2*i].find_all('small')

            if len(tn15_small) == 3:
                review['useful'] = int(tn15_small[0].get_text().split(' ')[0])
                review['useful_of'] = int(tn15_small[0].get_text().split(" ")[3])
                review['region'] = tn15_small[1].get_text()[5:]
                review['date'] = tn15_small[2].get_text()

            elif len(tn15_small) == 1:
                review['date'] = tn15_small[0].get_text()
                review['useful'] = None
                review['useful_of'] = None
                review['region'] = None

            elif len(tn15_small) == 2:
                review['date'] = tn15_small[1].get_text()
                if tn15_small[0].get_text()[0] == u'f':
                    review['region'] = tn15_small[0].get_text()[5:]
                    review['useful'] = None
                    review['useful_of'] = None
                else:
                    review['region'] = None
                    review['useful'] = int(tn15_small[0].get_text().split(' ')[0])
                    review['useful_of'] = int(tn15_small[0].get_text().split(' ')[3])

            movieReviews = movieReviews.append([review], ignore_index=True)
    return movieReviews


In [9]:
imdb = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 'numOfReviews',
                               'review_name', 'author', 'region', 'date', 'review_text'])


In [19]:
#get reviews from 0 to 99
for i in xrange(5,6):
    print i
    imdb = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 'numOfReviews',
                               'review_name', 'author', 'region', 'date', 'review_text'])
    
    imdb = imdb.append(ParseMovie(movieList.id[i]))
    cvs_name = './reviews_' + str(i) +'.csv'
    imdb.to_csv(cvs_name, encoding='utf-8', index=False)

5
Schindler's List 1375 ---------------
0.0
0.00727272727273
0.0145454545455
0.0218181818182
0.0290909090909
0.0363636363636
0.0436363636364
0.0509090909091
0.0581818181818
0.0654545454545
0.0727272727273
0.08
0.0872727272727
0.0945454545455
0.101818181818
0.109090909091
0.116363636364
0.123636363636
0.130909090909
0.138181818182
0.145454545455
0.152727272727
0.16
0.167272727273
0.174545454545
0.181818181818
0.189090909091
0.196363636364
0.203636363636
0.210909090909
0.218181818182
0.225454545455
0.232727272727
0.24
0.247272727273
0.254545454545
0.261818181818
0.269090909091
0.276363636364
0.283636363636
0.290909090909
0.298181818182
0.305454545455
0.312727272727
0.32
0.327272727273
0.334545454545
0.341818181818
0.349090909091
0.356363636364
0.363636363636
0.370909090909
0.378181818182
0.385454545455
0.392727272727
0.4
0.407272727273
0.414545454545
0.421818181818
0.429090909091
0.436363636364
0.443636363636
0.450909090909
0.458181818182
0.465454545455
0.472727272727
0.48
0.487272727273

In [237]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
import numpy as np

In [244]:
cv_inst = CountVectorizer()
a = cv_inst.fit_transform(imdb.review_text)
result = a.toarray()

In [245]:
#выкинем слова, которые встречаются < 100 раз
result = result[:, [i for i in range(result.shape[1]) if (200 > result[:, i].sum() and result[:, i].sum() > 100)]]
result.shape

(1375L, 174L)

In [252]:
lnreg = LinearRegression()

imdb.rating = imdb.rating.fillna(5)
y = imdb.rating.astype(int).values
np.random.shuffle(y)

lnreg.fit(result[:700], y[:700])
pred = lnreg.predict(result[700:]) 

ar_floor = np.apply_along_axis(np.floor, 0, pred)
ar_floor = np.where(ar_floor < 0, 0, ar_floor)
ar_floor = np.where(ar_floor > 10, 10, ar_floor)

((ar_floor - y[700:]) ** 2).mean() ** 0.5

2.8844410203711917

Etalon

In [248]:
((y - y.mean()) ** 2).mean() ** 0.5

2.4935747912724455

In [251]:
((ar_floor - y[700:]) ** 2).mean() ** 0.5

2.8692494824592356

In [223]:
ar_floor[:50], y[700:][:50]

(array([  8.,   8.,  10.,   3.,   3.,   7.,   9.,   8.,  10.,   8.,   7.,
          8.,   5.,   5.,  10.,   7.,  10.,   9.,  10.,   6.,   8.,   7.,
          3.,   7.,   7.,   9.,  10.,   7.,   3.,   6.,   6.,   8.,   8.,
          4.,   7.,  10.,   9.,   8.,   8.,   7.,   7.,  10.,   8.,   9.,
          9.,   6.,   4.,  10.,   6.,   6.]),
 array([ 9, 10,  5, 10,  5,  5, 10,  8,  9,  9, 10, 10, 10,  7, 10,  5,  9,
        10,  6,  5,  5, 10, 10, 10,  6, 10,  6, 10, 10, 10,  7,  5,  8,  5,
         5, 10,  9,  9, 10,  9,  5, 10, 10, 10, 10,  9, 10, 10,  9,  9]))

In [143]:
imdb = pd.DataFrame(columns = ['review_id', 'rating', 'useful', 'useful_of', 'movie_name', 'movie_id', 
                               'numOfReviews', 'review_name', 'author', 'region', 'date', 'review_text'])

#5, 23, 182
for i in xrange(24,181):
    cvs_name = './reviews/reviews_' + str(i) + '.csv'
    print cvs_name
    imdb1 = pd.read_csv(cvs_name)
    imdb = imdb.append(imdb1, ignore_index=True)

./reviews/reviews_24.csv
./reviews/reviews_25.csv
./reviews/reviews_26.csv
./reviews/reviews_27.csv
./reviews/reviews_28.csv
./reviews/reviews_29.csv
./reviews/reviews_30.csv
./reviews/reviews_31.csv
./reviews/reviews_32.csv
./reviews/reviews_33.csv
./reviews/reviews_34.csv
./reviews/reviews_35.csv
./reviews/reviews_36.csv
./reviews/reviews_37.csv
./reviews/reviews_38.csv
./reviews/reviews_39.csv
./reviews/reviews_40.csv
./reviews/reviews_41.csv
./reviews/reviews_42.csv
./reviews/reviews_43.csv
./reviews/reviews_44.csv
./reviews/reviews_45.csv
./reviews/reviews_46.csv
./reviews/reviews_47.csv
./reviews/reviews_48.csv
./reviews/reviews_49.csv
./reviews/reviews_50.csv
./reviews/reviews_51.csv
./reviews/reviews_52.csv
./reviews/reviews_53.csv
./reviews/reviews_54.csv
./reviews/reviews_55.csv
./reviews/reviews_56.csv
./reviews/reviews_57.csv
./reviews/reviews_58.csv
./reviews/reviews_59.csv
./reviews/reviews_60.csv
./reviews/reviews_61.csv
./reviews/reviews_62.csv
./reviews/reviews_63.csv


In [117]:
imdb1 = pd.read_csv('./reviews_5.csv')
imdb1

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [120]:
df5 = ParseMovie('tt0108052')

Schindler's List 1375 ---------------
0.0
0.00727272727273
0.0145454545455
0.0218181818182
0.0290909090909
0.0363636363636
0.0436363636364
0.0509090909091
0.0581818181818
0.0654545454545
0.0727272727273
0.08
0.0872727272727
0.0945454545455
0.101818181818
0.109090909091
0.116363636364
0.123636363636
0.130909090909
0.138181818182
0.145454545455
0.152727272727
0.16
0.167272727273
0.174545454545
0.181818181818
0.189090909091
0.196363636364
0.203636363636
0.210909090909
0.218181818182
0.225454545455
0.232727272727
0.24
0.247272727273
0.254545454545
0.261818181818
0.269090909091
0.276363636364
0.283636363636
0.290909090909
0.298181818182
0.305454545455
0.312727272727
0.32
0.327272727273
0.334545454545
0.341818181818
0.349090909091
0.356363636364
0.363636363636
0.370909090909
0.378181818182
0.385454545455
0.392727272727
0.4
0.407272727273
0.414545454545
0.421818181818
0.429090909091
0.436363636364
0.443636363636
0.450909090909
0.458181818182
0.465454545455
0.472727272727
0.48
0.487272727273
0